In [1]:
import os,pickle
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import torch
from tqdm import tqdm
from transformers import BitsAndBytesConfig,LlamaTokenizer,LlamaForCausalLM 
from Custom_bert_generate2 import CustomDataset, Model,get_generated_sentence_data, get_accuracy_by_bert
from get_model import get_model
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from utils.get_keywords import get_keywords_from_reference
from flair.models import TextClassifier
from flair.data import Sentence
from sklearn.metrics import accuracy_score

import transformers
from torch.utils.data import Dataset,DataLoader
from transformers import BertTokenizerFast, BertModel
from torch import nn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
device = 'cuda'
model_names = ['LLaMA-2-7B-Chat','LLaMA-3-7B-Instruct','Alpaca-7B',
               'DialoGPT-large', 'GPT-2-large',
               'WizardLM-2-7B','WizardLM-7B','WizardCoder-7B','WizardMath-7B',
              'ChatGLM-6B','Guanaco-2-7B']
model_name = model_names[8]

tokenizer,model = get_model(model_name,device)
model.eval()


In [2]:
def get_data():
    with open('/home/wooseok/peft/FT_input_datasets_llama.pickle', 'rb') as fr:
        data = pickle.load(fr)
    del_index = []
    for i in range(len(data)):
        if len(data['context'][i]) %2 ==0:
            del_index.append(i)
    data.drop(del_index, axis=0, inplace=True)
    data=data.reset_index(drop = True)
    return data

if model_name == 'Guanaco-2-7B':
    def make_prompt(j,attribute,sys_prompt,data):
        default_prompt = "### Instruction:"+"\n"
        sys_prompt = default_prompt+ sys_prompt + attribute+"\n"
        prompt = sys_prompt 
        conversation_history = ''
        for i in range(len(data['context'][j])):
            if i%2 == 0:
                utter2 = 'USER: '+data['context'][j][i]+"\n"
            if i%2 == 1:
                utter2 = 'ASSISTANT: '+data['context'][j][i]
            conversation_history = conversation_history + utter2
        prompt = prompt+"### Input: "+"\n"+conversation_history+"\n### Response:" +"\nASSISTANT: "
        return prompt, conversation_history
if model_name in ['WizardLM-2-7B','WizardLM-7B']:
    def make_prompt(j,attribute,sys_prompt,data):
        default_prompt = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."
        sys_prompt = default_prompt+ sys_prompt + attribute
        prompt = sys_prompt 
        conversation_history = ''
        for i in range(len(data['context'][j])):
            if i%2 == 0:
                utter2 = 'USER: '+data['context'][j][i]
            if i%2 == 1:
                utter2 = 'ASSISTANT: '+data['context'][j][i]+"</s>"
            conversation_history = conversation_history + utter2
        prompt = prompt+conversation_history +"ASSISTANT: "
        return prompt, conversation_history
       
if model_name in ['DialoGPT-large', 'GPT-2-large']:
    def make_prompt(j,attribute,sys_prompt,data):
        sys_prompt = sys_prompt +"\n" + attribute+"<|endoftext|>"+"\n"
        utter1 = data['context'][j][0] 
        prompt = sys_prompt +'Conversation:'
        conversation_history = ''
        for i in range(len(data['context'][j])):
            utter2 = '\n'+data['context'][j][i] +"\n"
            conversation_history = conversation_history + utter2
        prompt = prompt+conversation_history+"<|endoftext|>"
        return prompt, conversation_history

if model_name == 'LLaMA-3-7B-Instruct':

    def make_prompt(j,attribute,sys_prompt,data):
        sys_prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"+sys_prompt +attribute
        sys_prompt = sys_prompt +"<|eot_id|><|start_header_id|>user1<|end_header_id|>\n"
        
        conversation_history = ''
        for i in range(len(data['context'][j])):
            if i%2 == 0:
                utter2 = '\n\n'+data['context'][j][i] +'<|eot_id|><|start_header_id|>user2<|end_header_id|>'
            if i%2 == 1:
                utter2 = '\n\n'+data['context'][j][i] +'<|eot_id|><|start_header_id|>user1<|end_header_id|>'
            conversation_history = conversation_history + utter2
        prompt = sys_prompt+conversation_history
        
        return prompt, conversation_history
if model_name == 'LLaMA-2-7B-Chat':
    def make_prompt(j,attribute,sys_prompt,data):    
        sys_prompt = "[INST]<<SYS>>\n"+sys_prompt +"\n<</SYS>>\n" + attribute+"\n"+'Conversation:'
         
        
        conversation_history = ''
        for i in range(len(data['context'][j])):
            utter2 = '\n'+data['context'][j][i] +'</s>'
            conversation_history = conversation_history + utter2
        prompt = sys_prompt+conversation_history +'[/INST]'
        return prompt, conversation_history
if model_name in ['Alpaca-7B','WizardCoder-7B','WizardMath-7B','ChatGLM-6B']:
    def make_prompt(j,attribute,sys_prompt,data):
        default_prompt = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."

        default_prompt = default_prompt +"\n### Instruction:\n"
        sys_prompt = default_prompt + sys_prompt + attribute +"\n"+"\n### Input:"
        
        prompt = sys_prompt
        conversation_history = ''
        for i in range(len(data['context'][j])):
            if i%2 == 0:
                utter2 = '\n'+data['context'][j][i] +'</s>'
            if i%2 == 1:
                utter2 = '\n'+data['context'][j][i] +'</s>'
            conversation_history = conversation_history + utter2
        prompt = prompt+conversation_history +"\n\n### Response:"
        
        return prompt, conversation_history
def prompt_make(attribute,sys_prompt):
    data = get_data()
    
    prompt_gen = []
    history_gen = []
    for j in tqdm(range(len(data))):
        kk = make_prompt(j,attribute,sys_prompt,data)
        prompt_gen.append(kk[0])
        history_gen.append(kk[1])
                
    data['prompt']=prompt_gen
    data['history']=history_gen
    return data

In [3]:
def get_generated_sentences_data(attribute, sys_prompt,data_num,emotion,input_label):
    
    data = prompt_make(attribute,sys_prompt)
    data = data.sample(frac=1,random_state = 2).reset_index(drop=True)[:data_num]
    generated_sentences_masking_attribute = []
    generated_sentences = []
    
    for i in tqdm(range(len(data))):
        print(data['prompt'][i])
        
        encoded_context = torch.tensor([tokenizer.encode(data['prompt'][i],add_special_tokens= False)])
        eeencoded_context = encoded_context.squeeze()[:-1].unsqueeze(0)
        with torch.no_grad():
            outputs = model(eeencoded_context, output_hidden_states=True,use_cache=True)
        past_key_values = outputs['past_key_values']
        
        past_length = past_key_values[0][0].shape[2]
        eencoded_context = encoded_context.squeeze()[-1].unsqueeze(0).unsqueeze(0)
        attention_mask = torch.ones(1, past_length + len(eencoded_context[0]), dtype=torch.long)
        generated_ids = model.generate(eencoded_context.to(device),
                                       attention_mask=attention_mask.to(device),
                                       past_key_values=past_key_values,
                                       max_new_tokens=100,
                                       pad_token_id=tokenizer.eos_token_id,
                                       output_logits=True,
                                       return_dict_in_generate=True)
        generated_sentence= tokenizer.decode(generated_ids.sequences[:, eencoded_context.shape[-1]:][0], skip_special_tokens=True)
        if model_name in ['WizardLM-2-7B','WizardLM-7B']:
            first_user_index = generated_sentence.find('USER')
    
            # 첫 번째 'USER'가 등장하기 전까지의 부분 문자열 추출
            generated_sentence = generated_sentence[:first_user_index]
        
        #print(generated_ids)
        #generated_sentence= tokenizer.decode(generated_ids.sequences[0], skip_special_tokens=True)

        generated_sentences.append(generated_sentence)
        print("########GENERATED:########")
        print(generated_sentence)
        print("########Generation 끝#### ")
    
    data['generated'] =generated_sentences
    data['Emotion'] = input_label
    with open("최종results_{}/generatedSentence_{}.pkl".format(model_name,emotion),"wb") as f:
        pickle.dump(data, f)
    return data

In [4]:
if model_name in ['DialoGPT-large', 'GPT-2-large','WizardMath-7B','ChatGLM-6B']:
    def get_generated_sentences_data(attribute, sys_prompt,data_num,emotion,input_label):
        
        data = prompt_make(attribute,sys_prompt)
        data = data.sample(frac=1,random_state = 1).reset_index(drop=True)[:data_num]
        generated_sentences_masking_attribute = []
        generated_sentences = []
        
        for i in tqdm(range(len(data))):
            print(data['prompt'][i])
            encoded_context = torch.tensor([tokenizer.encode(data['prompt'][i])]).to(device)
            attention_mask = torch.ones(1,len(encoded_context[0])).to(device)
            generated_ids = model.generate(encoded_context, max_length=256, do_sample =True,
                                           attention_mask = attention_mask,
                                           #pad_token_id=tokenizer.eos_token_id,
                                           
                                          )
            generated_sentence= tokenizer.decode(generated_ids[:, encoded_context.shape[-1]:][0], skip_special_tokens=True)
            generated_sentences.append(generated_sentence)
            print("########GENERATED:########")
            print(generated_sentence)
            print(" ")
            
        data['generated'] =generated_sentences
        data['Emotion'] = input_label
        with open("최종results_{}/generatedSentence_{}.pkl".format(model_name,emotion),"wb") as f:
            pickle.dump(data, f)
        return data

In [5]:
sys_prompt = "In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context."
data_num = 50

In [7]:
emotions = ["joy", "surprise", "anger", "sadness", "disgust", "fear","general"]
attribute_name_list = ["You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is joy.",
                       "You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is surprise.",
                      "You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is anger.",
                        "You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is sadness.",
                        "You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is disgust.",                    
                        "You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is fear.",
                       ""
                      ]
for i in range(len(attribute_name_list)):
    attribute = attribute_name_list[i]
    get_generated_sentences_data(attribute, sys_prompt,data_num,emotions[i],input_label=i)



  0%|                                                                                  | 0/50 [00:00<?, ?it/s]


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is joy.

### Input:
Mom , when you come home , please buy a coat , a skirt , a sweater ... </s>

### Response:


NameError: name 'tokenizer' is not defined

In [8]:
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


In [9]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        
        self.classifier  = nn.Sequential(
            nn.ReLU(),
            nn.Linear(768,class_num))
    def forward(self,inputs):
        bert_outputs = self.bert(**inputs,return_dict =True)
        pooler_output = bert_outputs.last_hidden_state[:,0]
        
        logits = self.classifier(pooler_output)        
        return logits


class CustomDataset(Dataset):
    def __init__(self, df):
        self.data =df 
        self.data_list, self.label_list = self.load_data()
        
    def __len__(self):
        return len(self.label_list)
    def load_data(self):
        data_list = self.data['generated']
        label_list = self.data['Emotion']
        
        return data_list, label_list 
    def __getitem__(self, index):
        data = self.data_list[index]
        label = torch.tensor(self.label_list[index], dtype = torch.long)
        return data, label

def get_accuracy_by_bert(data,device,
                         emotion,class_num
                        ):
    if emotion =='ALL':
        bert_model = torch.load('/home/wooseok/layer_analysis_prompt_based/Bert_emotion_classifier/6classbert_model.pt')

    else:
        bert_model = torch.load('/home/wooseok/layer_analysis_prompt_based/Bert_emotion_classifier/2class/{}_bert_model.pt'.format(emotion))

    bert_model.to(device)
    bert_model.eval()

    max_len= 500
    
    data = data[['generated','Emotion']]
    test_set = CustomDataset(data)
    test_dl = DataLoader(test_set,batch_size=20, shuffle = False)    
    
    test_acc = 0.0
    test_n_samples = 0
    
    all_y_true = []
    all_y_pred = []
    with torch.no_grad():
        for batch_idx,batch in enumerate(tqdm(test_dl, ncols=80, desc='valid_step')):
            data, y_true = batch
            data = bert_tokenizer(list(data), return_tensors='pt', padding=True, truncation=True)
            
            data = { k: v.to(device) for k, v in data.items() }
            y_true = y_true.to(device)
            all_y_true.extend(y_true.cpu().numpy())
            
            y_pred = bert_model(data)
            y_preddd = y_pred.argmax(dim=1)
            all_y_pred.extend(y_preddd.cpu().numpy())
            y_prediction = y_pred.argmax(1)

            
            test_acc += torch.sum(y_prediction == y_true).item()
            test_n_samples += len(y_true)
    test_acc = (test_acc / test_n_samples) * 100.
    #print(y_pred.argmax(1))
    print("accuracy:",test_acc)
    return test_acc  ,all_y_true,all_y_pred,test_n_samples

In [10]:
class_num = 2       
bert_model = Model()


with open("최종results_{}/generatedSentence_{}.pkl".format(model_name,"general"),"rb") as f:
    data_general =pickle.load(f)

binary_general_values =[]
binary_controlled_values =[]
for i in range(len(emotions)-1):
    with open("최종results_{}/generatedSentence_{}.pkl".format(model_name,emotions[i]),"rb") as f:
        data_att =pickle.load(f)
    data_att['Emotion']=0
    data_general['Emotion']= 0
    binary_general_values.append(get_accuracy_by_bert(data_general,device,emotions[i],class_num=2)[0])
    binary_controlled_values.append(get_accuracy_by_bert(data_att,device,emotions[i],class_num=2)[0]) 
with open("최종results_{}/accuracy_binary_general.pkl".format(model_name),"wb") as f:
        pickle.dump(binary_general_values, f)
with open("최종results_{}/accuracy_binary_controlled.pkl".format(model_name),"wb") as f:
        pickle.dump(binary_controlled_values, f)

print(" ")
print(np.mean(binary_general_values))
print(np.mean(binary_controlled_values))

valid_step: 100%|█████████████████████████████████| 3/3 [00:00<00:00,  5.34it/s]


accuracy: 60.0


valid_step: 100%|█████████████████████████████████| 3/3 [00:00<00:00, 11.10it/s]


accuracy: 86.0


valid_step: 100%|█████████████████████████████████| 3/3 [00:00<00:00,  9.70it/s]


...

accuracy: 32.0


valid_step: 100%|█████████████████████████████████| 3/3 [00:00<00:00,  9.64it/s]


accuracy: 4.0


valid_step: 100%|█████████████████████████████████| 3/3 [00:00<00:00, 10.67it/s]

accuracy: 32.0
 
29.666666666666668
49.333333333333336


In [11]:

class_num = 6
bert_model = Model()

with open("최종results_{}/generatedSentence_{}.pkl".format(model_name,"joy"),"rb") as f:
        data_joy = pickle.load(f)
with open("최종results_{}/generatedSentence_{}.pkl".format(model_name,"surprise"),"rb") as f:
        data_surprise=pickle.load(f)
with open("최종results_{}/generatedSentence_{}.pkl".format(model_name,"anger"),"rb") as f:
        data_anger=pickle.load(f)
with open("최종results_{}/generatedSentence_{}.pkl".format(model_name,"sadness"),"rb") as f:
        data_sadness=pickle.load(f)
with open("최종results_{}/generatedSentence_{}.pkl".format(model_name,"disgust"),"rb") as f:
        data_disgust=pickle.load(f)
with open("최종results_{}/generatedSentence_{}.pkl".format(model_name,"fear"),"rb") as f:
        data_fear=pickle.load(f)

data_joy['Emotion']=0
data_surprise['Emotion']=1
data_anger['Emotion']=2
data_sadness['Emotion']=3
data_disgust['Emotion']=4
data_fear['Emotion']=5

multi_controlled_values =[]
multi_controlled_values.append(get_accuracy_by_bert(data_joy,device,'ALL',6)[0])  # 0 
multi_controlled_values.append(get_accuracy_by_bert(data_surprise,device,'ALL',6)[0]) #1
multi_controlled_values.append(get_accuracy_by_bert(data_anger,device,'ALL',6)[0]) # 2
multi_controlled_values.append(get_accuracy_by_bert(data_sadness,device,'ALL',6)[0]) # 3 
multi_controlled_values.append(get_accuracy_by_bert(data_disgust,device,'ALL',6)[0]) # 4 
multi_controlled_values.append(get_accuracy_by_bert(data_fear,device,'ALL',6)[0]) # 5
with open("최종results_{}/accuracy_multi_controlled.pkl".format(model_name),"wb") as f:
        pickle.dump(multi_controlled_values, f)
print(np.mean(multi_controlled_values))

valid_step: 100%|█████████████████████████████████| 3/3 [00:00<00:00, 10.07it/s]


accuracy: 56.00000000000001


valid_step: 100%|█████████████████████████████████| 3/3 [00:00<00:00, 11.20it/s]


accuracy: 50.0


valid_step: 100%|█████████████████████████████████| 3/3 [00:00<00:00, 10.13it/s]


...

accuracy: 40.0


valid_step: 100%|█████████████████████████████████| 3/3 [00:00<00:00, 10.82it/s]


accuracy: 26.0


valid_step: 100%|█████████████████████████████████| 3/3 [00:00<00:00, 10.54it/s]

accuracy: 40.0
37.0
